## 1st practice cifar10

In [16]:
import matplotlib.pyplot as plt
import keras
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Input, Conv2D, MaxPool2D, BatchNormalization, ReLU, Flatten, Dense, Add, Dropout, Activation
from tensorflow.keras.optimizers import Adam
import tensorflow as tf
from keras.datasets import cifar10

In [2]:
#データのロード
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

#０〜１へ正則化
X_train = X_train.astype("float32")/255.0
X_test = X_test.astype("float32")/255.0

# 目的変数のone-hotエンコーディング
Y_train = keras.utils.to_categorical(y_train)
Y_test = keras.utils.to_categorical(y_test)

input_shape = X_train.shape[1:]
train_samples = X_train.shape[0]
test_samples = X_test.shape[0]


In [6]:
print(X_train.shape)
print(Y_train.shape)
print(X_test.shape)
print(Y_test.shape)

(50000, 32, 32, 3)
(50000, 10)
(10000, 32, 32, 3)
(10000, 10)


In [12]:
num_classes = 10

model = Sequential()

#CNN layer1
model.add(Conv2D(32, (3,3), padding="same", #3✖️3のフィルタを32枚用意し，outputが32枚になるようにする
                 input_shape=X_train.shape[1:]))#padding=sameによってoutputはinputと同じサイズになる
model.add(Activation("relu"))
model.add(Conv2D(32, (3, 3)))
model.add(Activation("relu"))
model.add(MaxPool2D(pool_size=(2,2)))#2✖️2の領域で最大の数値を出力する
model.add(Dropout(0.25))#過学習の防止

# CNN layer 2
model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

# output
model.add(Flatten()) #output shapeが一次元になっている
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

In [14]:
print(model.summary())

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
activation_2 (Activation)    (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 30, 30, 32)        9248      
_________________________________________________________________
activation_3 (Activation)    (None, 30, 30, 32)        0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 15, 15, 32)        0         
_________________________________________________________________
dropout (Dropout)            (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 15, 15, 64)       

In [17]:
model.compile(
    loss = "categorical_crossentropy",
    optimizer = Adam(lr=0.001),
    metrics = ["accuracy"]
)

In [19]:
fit = model.fit(X_train, Y_train,
              batch_size=128,
              epochs=10,
              verbose=1,
              validation_split=0.1 # 今回は訓練データセットの10%をvalidationデータセットとして使う
                )

Epoch 1/10
352/352 [==============================] - 222s 632ms/step - loss: 1.5898 - accuracy: 0.4201 - val_loss: 1.2521 - val_accuracy: 0.5586
Epoch 2/10
352/352 [==============================] - 167s 473ms/step - loss: 1.2103 - accuracy: 0.5691 - val_loss: 1.0169 - val_accuracy: 0.6394
Epoch 3/10
352/352 [==============================] - 128s 362ms/step - loss: 1.0367 - accuracy: 0.6304 - val_loss: 0.8963 - val_accuracy: 0.6862
Epoch 4/10
352/352 [==============================] - 127s 362ms/step - loss: 0.9301 - accuracy: 0.6736 - val_loss: 0.8203 - val_accuracy: 0.7164
Epoch 5/10
352/352 [==============================] - 125s 354ms/step - loss: 0.8509 - accuracy: 0.7012 - val_loss: 0.7475 - val_accuracy: 0.7374
Epoch 6/10
352/352 [==============================] - 124s 353ms/step - loss: 0.7870 - accuracy: 0.7252 - val_loss: 0.7065 - val_accuracy: 0.7534
Epoch 7/10
352/352 [==============================] - 2035s 6s/step - loss: 0.7374 - accuracy: 0.7407 - val_loss: 0.6990 - v

In [22]:
from tabnanny import verbose


score = model.evaluate(X_test, Y_test,
                 verbose=0
                 )

print("Test score:", score[0])
print("Test accuracy:", score[1])

Test score: 0.6683177351951599
Test accuracy: 0.7692999839782715


# 2nd practice（Build your own CNN）

In [29]:
num_classes = 10

model = Sequential()

#CNN layer1
model.add(Conv2D(32, (3,3), padding="same", #3✖️3のフィルタを32枚用意し，outputが32枚になるようにする
                 input_shape=X_train.shape[1:]))#padding=sameによってoutputはinputと同じサイズになる
model.add(Activation("relu"))
model.add(Conv2D(32, (3, 3)))
model.add(BatchNormalization(axis=1))
model.add(Activation("relu"))
model.add(MaxPool2D(pool_size=(2,2)))#2✖️2の領域で最大の数値を出力する
model.add(Dropout(0.25))#過学習の防止

# CNN layer 2
model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(BatchNormalization(axis=1))
model.add(Activation('relu'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

#CNN layer 3
model.add(Conv2D(128, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(128, (3, 3)))
model.add(BatchNormalization(axis=1))
model.add(Activation('relu'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

# output
model.add(Flatten()) #output shapeが一次元になっている
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

In [30]:
model.compile(
    loss = "categorical_crossentropy",
    optimizer = Adam(lr=0.01),
    metrics = ["accuracy"]
)

In [31]:
fit = model.fit(X_train, Y_train,
              batch_size=128,
              epochs=10,
              verbose=1,
              validation_split=0.1 # 今回は訓練データセットの10%をvalidationデータセットとして使う
                )

Epoch 1/10
 11/352 [..............................] - ETA: 4:16 - loss: 4.3270 - accuracy: 0.1080

KeyboardInterrupt: 